In [ ]:
import numpy as np
import itertools

In [ ]:
def manual_significance(S,Bs,se,bes):
    return (S * se) / np.sqrt(S * se + sum([B * be for B, be in zip(Bs, bes)]))

In [ ]:
def points2hrs(points):
    points2sec = 126 / (5**3 * 6)
    return points * points2sec * 3600**-1

In [ ]:
def maxhrs2points(maxhrs):
    points2sec = 126 / (5**3 * 6)
    return maxhrs * 3600 * points2sec**-1

In [ ]:
def find_sig_dep(signal, backgrounds, numS, numBs, cuts):
    """Each row of cuts should take the form [index, isCutBelow, vals]
    where 'index' gives the index of the feature being cut on
    in the signal/background data, 'isCutBelow' is a boolean specifying
    if that variable should involve removing data points below (True) or
    above (False) the given values, and 'vals' is a list of values to cut at"""
    
    indices = [row[0] for row in cuts]
    isCutBelows = [row[1] for row in cuts]
    points = list(itertools.product(*[row[-1] for row in cuts]))
    sigsize = len(signal)
    bgsizes = [len(bg) for bg in backgrounds]
    sigs = []
    sigsizeprimes = []
    bgsizesprimes = []
    for point in points:
        sigsizeprime = np.count_nonzero(np.logical_and.reduce(
            np.array([np.where(signal[:,index] > val, isCutBelow, not isCutBelow) 
                      for index, isCutBelow, val in zip(indices, isCutBelows, point)])))
        bgsizesprime = np.array([np.count_nonzero(np.logical_and.reduce(
            np.array([np.where(background[:,index] > val, isCutBelow, not isCutBelow) 
                      for index, isCutBelow, val in zip(indices, isCutBelows, point)]))) for background in backgrounds])
        sigsizeprimes.append(sigsizeprime)
        bgsizesprimes.append(bgsizesprime)
        sigs.append(manual_significance(numS, numBs, sigsizeprime/sigsize, bgsizesprime/bgsizes))
    return [points, sigs, sigsizeprimes, bgsizesprimes]

In [ ]:
sigss = signal_data
bgss = background_data
names = get47Dfeatures()
masses, sig_css, bg_css = get_elijah_ttbarzp_cs()
zp_cs = cross_section_helper(masses, sig_css, bg_css, mass_units='GeV')
masses = [350, 500, 1000, 2000, 4000]
sig_css = [zp_cs.sig_cs(mass) for mass in masses]
conv = 10**15 / 10**12 # conv * lumi (in fb^{-1}) * cross sec (in pb) = # of events
lumi = 3000
signal_yields = np.array([conv * lumi * sig_cs for sig_cs in sig_css])
background_yields = np.array([conv * lumi * bg_cs for bg_cs in bg_css])

In [ ]:
for signal_yield in signal_yields:
    print(manual_significance(signal_yield, background_yields, 1, [1]*3))

7.248418610012742
2.757897361779127
0.2855952155740879
0.014759138609928545
0.000260774522132171


In [ ]:
time_before = datetime.now().minute * 60 + datetime.now().second
cuts = [[names.index('M b1 b2'), True, [0, 100, 200, 300, 400]],
        [names.index('M b1 b3'), True, [0, 100, 200, 300, 400]],
        [names.index('M j1 j2'), False, [100, 200, 300, 400, 10000]],
        [names.index('pT b1'), True, [0, 100, 200, 300, 400, 500]]]
points, sigs, sigsizeprimes, bgsizesprimes = find_sig_dep(sigss[0], bgss, signal_yields[0], background_yields, cuts)
time_after = datetime.now().minute * 60 + datetime.now().second
print(f"Runtime: {time_after - time_before} seconds")

1000000 [1000000, 1000000, 1000000]
Runtime: 134 seconds


In [ ]:
print(max(sigs))
print(points[sigs.index(max(sigs))])

20.5630757507069
(300, 200, 200, 200)


In [ ]:
time_before = datetime.now().minute * 60 + datetime.now().second
cuts = [[names.index('M b1 b2'), True, [200, 300, 400, 500, 600]],
        [names.index('M b1 b3'), True, [0, 100, 200, 300, 400]],
        [names.index('M j1 j2'), False, [100, 200, 300, 400, 10000]],
        [names.index('pT b1'), True, [0, 100, 200, 300, 400, 500]]]
points, sigs, sigsizeprimes, bgsizesprimes = find_sig_dep(sigss[1], bgss, signal_yields[1], background_yields, cuts)
time_after = datetime.now().minute * 60 + datetime.now().second
print(f"Runtime: {time_after - time_before} seconds")

Runtime: 127 seconds


In [ ]:
print(max(sigs))
print(points[sigs.index(max(sigs))])

12.293874578677672
(400, 300, 200, 300)


In [ ]:
time_before = datetime.now().minute * 60 + datetime.now().second
cuts = [[names.index('M b1 b2'), True, [600, 700, 900, 1100, 1200]],
        [names.index('M b1 b3'), True, [0, 100, 200, 300, 400]],
        [names.index('M j1 j2'), False, [100, 200, 300, 400, 10000]],
        [names.index('pT b1'), True, [200, 300, 400, 500, 600]]]
points, sigs, sigsizeprimes, bgsizesprimes = find_sig_dep(sigss[2], bgss, signal_yields[2], background_yields, cuts)
time_after = datetime.now().minute * 60 + datetime.now().second
print(f"Runtime: {time_after - time_before} seconds")

Runtime: 102 seconds


In [ ]:
print(max(sigs))
print(points[sigs.index(max(sigs))])

6.22708730618894
(900, 300, 200, 400)


In [ ]:
time_before = datetime.now().minute * 60 + datetime.now().second
cuts = [[names.index('M b1 b2'), True, [1200, 1500, 1800, 2000, 2300]],
        [names.index('M b1 b3'), True, [200, 300, 400, 500, 600]],
        [names.index('M j1 j2'), False, [100, 200, 300, 400, 10000]],
        [names.index('pT b1'), True, [600, 700, 800, 900, 1000]]]
points, sigs, sigsizeprimes, bgsizesprimes = find_sig_dep(sigss[3], bgss, signal_yields[3], background_yields, cuts)
time_after = datetime.now().minute * 60 + datetime.now().second
print(f"Runtime: {time_after - time_before} seconds")

Runtime: 105 seconds


In [ ]:
print(max(sigs))
print(points[sigs.index(max(sigs))])

3.1672873562345023
(1800, 400, 200, 800)


In [ ]:
time_before = datetime.now().minute * 60 + datetime.now().second
cuts = [[names.index('M b1 b2'), True, [2000, 2500, 3000, 3500, 4000]],
        [names.index('M b1 b3'), True, [0, 10, 20, 50]],
        [names.index('M j1 j2'), False, [100, 200, 300, 400, 500]],
        [names.index('pT b1'), True, [0, 50, 100, 200, 300]]]
points, sigs, sigsizeprimes, bgsizesprimes = find_sig_dep(sigss[4], bgss, signal_yields[4], background_yields, cuts)
time_after = datetime.now().minute * 60 + datetime.now().second
print(f"Runtime: {time_after - time_before} seconds")

Runtime: 100 seconds


In [ ]:
print(max(sigs))
print(points[sigs.index(max(sigs))])

1.0127164593743987
(3500, 0, 200, 100)
